In [1]:
!pip install transformers

In [8]:
# Импорт необходимых библиотек
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

# Выбор устройства (GPU или CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Загрузка токенизатора и модели
tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2")
model = GPT2LMHeadModel.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2")
model.to(device)

def generate_prompt(traits):
    """
    Создает текстовый промпт на основе личностных показателей.
    
    Параметры
    ----------
    traits : dict
        Словарь с ключами, представляющими личностные характеристики (например,
        'extraversion', 'neuroticism', 'agreeableness', 'conscientiousness',
        'openness'), и значениями, описывающими степень выраженности каждой
        характеристики.
    
    Возвращает
    -------
    str
        Сформированный текстовый промпт, содержащий описание личностных черт и
        запрос на перечисление подходящих профессий для кандидата.
    
    Примечания
    ---------
    - Функция создает текстовый шаблон, который впоследствии используется для
      генерации рекомендаций на основе входных данных.
    - Включенные в шаблон характеристики помогают модели лучше понимать запрос
      и формировать релевантные результаты.
    """
    prompt = (
        f"У кандидата следующие показатели личности:\n"
        f"- Экстраверсия: {traits['extraversion']}\n"
        f"- Нейротизм: {traits['neuroticism']}\n"
        f"- Доброжелательность: {traits['agreeableness']}\n"
        f"- Сознательность: {traits['conscientiousness']}\n"
        f"- Открытость опыту: {traits['openness']}\n\n"
        f"На основе этих показателей перечислите наиболее подходящие профессии для кандидата. "
        f"Ответьте кратко, перечислив не более трех профессий."
    )
    return prompt

def generate_advice(traits, max_length=150):
    """
    Генерирует краткие рекомендации по работе на основе личностных показателей.
    
    Параметры
    ----------
    traits : list
        Список строк, представляющих личностные характеристики или черты,
        которые будут использоваться для создания текста рекомендации.
    max_length : int, optional
        Максимальная длина генерируемого текста в символах (по умолчанию 150).
    
    Возвращает
    -------
    str
        Сгенерированный текст с рекомендацией, содержащий описание действий
        или советов, основанных на личностных характеристиках. Текст возвращается
        в виде строки без специальных токенов, только основная часть.
    
    Примечания
    ---------
    - Для генерации текста используется предварительно обученная модель языковой
      генерации.
    - Функция обрабатывает промпт с помощью токенизатора и модели, которая
      генерирует текст по заданным параметрам.
    - Выходной текст очищается от исходного промпта и возвращается только
      сгенерированная часть.
    """
    prompt = generate_prompt(traits)
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)

    output = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=3,
        do_sample=True,
        top_k=10,
        top_p=0.8,
        temperature=0.5,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id
    )

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    # Удаляем исходный промпт из сгенерированного текста
    advice = generated_text[len(prompt):].strip()
    # Оставляем только первую часть до точки или переноса строки
    return advice




In [9]:
# Пример использования
if __name__ == "__main__":
    traits = {
        'extraversion': 0.8,
        'neuroticism': 0.2,
        'agreeableness': 0.7,
        'conscientiousness': 0.9,
        'openness': 0.6
    }

    advice = generate_advice(traits)
    print("Рекомендации по работе:")
    print(advice)

Рекомендации по работе:
Список профессий для кандидата в депутаты Государственной Думы

1. Кандидат в депутаты государственной Думы.
2. Кандидатура в депутаты.
3. Кандидата в депутаты в Государственной Думе.
4. Кандидаты в депутаты на выборах в Государственную Думу.
5. Кандидарты в депутаты, выдвинутые избирательными объедин
